In [1]:
from pathlib import Path
from typing import List

import keras
from keras.applications.resnet import ResNet50
from keras.applications.vgg19 import VGG19
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.layers import Activation, Conv2D, Concatenate
from keras.layers import Dense, Input, GlobalAveragePooling2D, Dropout, UpSampling2D
from keras.models import Model
from keras import backend as K

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cv2


2022-10-13 20:23:18.709196: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-13 20:23:19.079965: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-10-13 20:23:19.167726: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/zorkin/Yandex.Disk/Обучение/Hackathon/vladivostok_chempionat/model_train/venv/lib/python3.

## Датасет

создание датасета для загрузки

In [19]:
from get_dataset import EyeDataset
# Инициализируем датасет
dataset = EyeDataset("dataset/train_dataset_mc")

### Датасет преобразованных файлов 256*256 после аугументации

In [2]:
from get_dataset import AugDataset
# Инициализируем датасет
base_dataset = Path('dataset/train_dataset_mc')
aug_dataset = Path('dataset/augument')
dataset = AugDataset(base_dataset, aug_dataset)

In [3]:
nums = {'nums': [x for x in range(len(dataset))]}
dataset_df = pd.DataFrame(nums)

In [4]:
def train_validate_test_split(df, train_percent=.6, validate_percent=.2, seed=None):
    np.random.seed(seed)
    perm = np.random.permutation(df.index)
    m = len(df.index)
    train_end = int(train_percent * m)
    validate_end = int(validate_percent * m) + train_end
    train = df.iloc[perm[:train_end]]
    validate = df.iloc[perm[train_end:validate_end]]
    test = df.iloc[perm[validate_end:]]
    return train, validate, test

In [5]:
train_df, validate_df, _ = train_validate_test_split(dataset_df, 0.6, 0.4)
print(len(train_df) + len(validate_df))
len(dataset_df), len(train_df), len(validate_df)

65200


(65200, 39120, 26080)

In [9]:
state_size = 256

## Генератор карзины

In [6]:
def keras_generator(nums: pd.DataFrame, dataset: AugDataset, batch_size: int):
    while True:
        x_batch = []
        y_batch = []
        
        for i in range(batch_size):
            try:
                num = nums.sample(1).values[0][0]

                img = dataset[num]['image']
                mask = dataset[num]['mask']
                
                #img = cv2.resize(img, (state_size,state_size))
                #mask = cv2.resize(mask, (state_size,state_size))

                x_batch += [img]
                y_batch += [mask[..., 0]]
            except:
                continue

        x_batch = np.array(x_batch)
        y_batch = np.array(y_batch)

        yield x_batch, np.expand_dims(y_batch, -1)

In [7]:
for x, y in keras_generator(train_df, dataset, 16):
    break
x.shape, y.shape

((16, 256, 256, 3), (16, 256, 256, 1))

In [10]:
base_model = ResNet50(weights='imagenet', input_shape=(state_size,state_size,3), include_top=False)
base_out = base_model.output
base_out

2022-10-13 20:25:10.661123: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/zorkin/Yandex.Disk/Обучение/Hackathon/vladivostok_chempionat/model_train/venv/lib/python3.10/site-packages/cv2/../../lib64:
2022-10-13 20:25:10.661437: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-10-13 20:25:10.661454: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (astra): /proc/driver/nvidia/version does not exist
2022-10-13 20:25:10.661649: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebui

<KerasTensor: shape=(None, 8, 8, 2048) dtype=float32 (created by layer 'conv5_block3_out')>

In [11]:
base_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 262, 262, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 128, 128, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [12]:
inp = base_model.get_layer('input_1').output

conv1 = base_model.get_layer('conv1_relu').output # None, 128, 128, 64
conv2 = base_model.get_layer('conv2_block3_out').output # None, 64, 64, 256
conv3 = base_model.get_layer('conv3_block4_1_relu').output # None, 32, 32, 128
conv4 = base_model.get_layer('conv4_block1_2_relu').output # None, 16, 16, 256
conv5 = base_model.get_layer('conv5_block1_2_relu').output # None, 8, 8, 512

In [14]:
inp = base_model.get_layer('input_1').output

conv1 = base_model.get_layer('conv1_relu').output # None, 256, 256, 64
conv2 = base_model.get_layer('conv2_block2_1_relu').output # None, 128, 128, 64
conv3 = base_model.get_layer('conv3_block2_2_relu').output # None, 64, 64, 128
conv4 = base_model.get_layer('conv4_block2_1_relu').output # None, 32, 32, 256
conv5 = base_model.get_layer('conv5_block1_2_relu').output # None, 16, 16, 512


In [13]:
conv1, conv2, conv3, conv4, conv5

(<KerasTensor: shape=(None, 128, 128, 64) dtype=float32 (created by layer 'conv1_relu')>,
 <KerasTensor: shape=(None, 64, 64, 256) dtype=float32 (created by layer 'conv2_block3_out')>,
 <KerasTensor: shape=(None, 32, 32, 128) dtype=float32 (created by layer 'conv3_block4_1_relu')>,
 <KerasTensor: shape=(None, 16, 16, 256) dtype=float32 (created by layer 'conv4_block1_2_relu')>,
 <KerasTensor: shape=(None, 8, 8, 512) dtype=float32 (created by layer 'conv5_block1_2_relu')>)

In [16]:
up1 = UpSampling2D(2, interpolation='bilinear')(conv5)
conc_1 = Concatenate()([up1, conv4])
conv_conc_1 = Conv2D(512, (3,3), padding='same')(conc_1)
conv_conc_1 = Activation('relu')(conv_conc_1)

up2 = UpSampling2D(2, interpolation='bilinear')(conv_conc_1)
conc_2 = Concatenate()([up2, conv3])
conv_conc_2 = Conv2D(256, (3,3), padding='same')(conc_2)
conv_conc_2 = Activation('relu')(conv_conc_2)

up3 = UpSampling2D(2, interpolation='bilinear')(conv_conc_2)
conc_3 = Concatenate()([up3, conv2])
conv_conc_3 = Conv2D(128, (3,3), padding='same')(conc_3)
conv_conc_3 = Activation('relu')(conv_conc_3)

up4 = UpSampling2D(2, interpolation='bilinear')(conv_conc_3)
conc_4 = Concatenate()([up4, conv1])
conv_conc_4 = Conv2D(64, (3,3), padding='same')(conc_4)
conv_conc_4 = Activation('relu')(conv_conc_4)

up5 = UpSampling2D(2, interpolation='bilinear')(conv_conc_4)
conv_conc_5 = Conv2D(1, (3,3), padding='same')(up5)
result = Activation('sigmoid')(conv_conc_5)

In [14]:
up1 = UpSampling2D(2, interpolation='bilinear')(conv5)
conc_1 = Concatenate()([up1, conv4])
conv_conc_1 = Conv2D(256, (3,3), padding='same')(conc_1)
conv_conc_1 = Activation('relu')(conv_conc_1)

up2 = UpSampling2D(2, interpolation='bilinear')(conv_conc_1)
conc_2 = Concatenate()([up2, conv3])
conv_conc_2 = Conv2D(128, (3,3), padding='same')(conc_2)
conv_conc_2 = Activation('relu')(conv_conc_2)

up3 = UpSampling2D(2, interpolation='bilinear')(conv_conc_2)
conc_3 = Concatenate()([up3, conv2])
conv_conc_3 = Conv2D(64, (3,3), padding='same')(conc_3)
conv_conc_3 = Activation('relu')(conv_conc_3)

up4 = UpSampling2D(2, interpolation='bilinear')(conv_conc_3)
conc_4 = Concatenate()([up4, conv1])
conv_conc_4 = Conv2D(32, (3,3), padding='same')(conc_4)
conv_conc_4 = Activation('relu')(conv_conc_4)

up5 = UpSampling2D(2, interpolation='bilinear')(conv_conc_4)
conv_conc_5 = Conv2D(1, (3,3), padding='same')(up5)
result = Activation('sigmoid')(conv_conc_5)

In [15]:
result

<KerasTensor: shape=(None, 256, 256, 1) dtype=float32 (created by layer 'activation_4')>

In [16]:
model = Model(inputs=base_model.input, outputs=result)

In [22]:

best_w = keras.callbacks.ModelCheckpoint(
    'resnet_best.h5',
    monitor='val_loss',
    verbose=0,
    save_best_only=True,
    save_weights_only=True,
    mode='auto',
    period=1,
)

last_w = keras.callbacks.ModelCheckpoint(
    'resnet_last.h5',
    monitor='val_loss',
    verbose=0,
    save_best_only=False,
    save_weights_only=True,
    mode='auto',
    period=1,
)

callbacks = [best_w, last_w]

adam = keras.optimizers.Adam(
    learning_rate=0.0001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    name="Adam",
)


model.compile(adam, 'binary_crossentropy')

In [23]:
batch_size = 10

model.fit_generator(
    keras_generator(train_df, dataset, batch_size),
    steps_per_epoch=100,
    epochs=10,
    verbose=1,
    callbacks=callbacks,
    validation_data=keras_generator(validate_df, dataset, batch_size),
    validation_steps=40,
    class_weight=None,
    max_queue_size=10,
    workers=1,
    use_multiprocessing=False,
    shuffle=True,
    initial_epoch=0,
)

Epoch 1/10


/tmp/ipykernel_87182/2630890961.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


100/100 [==============================] - 159s 2s/step - loss: 0.0511 - val_loss: 0.0430
Epoch 2/10
100/100 [==============================] - 156s 2s/step - loss: 0.0502 - val_loss: 0.0505
Epoch 3/10
100/100 [==============================] - 156s 2s/step - loss: 0.0447 - val_loss: 0.0488
Epoch 4/10
100/100 [==============================] - 163s 2s/step - loss: 0.0465 - val_loss: 0.0505
Epoch 5/10
100/100 [==============================] - 159s 2s/step - loss: 0.0456 - val_loss: 0.0479
Epoch 6/10
100/100 [==============================] - 158s 2s/step - loss: 0.0408 - val_loss: 0.0440
Epoch 7/10
100/100 [==============================] - 158s 2s/step - loss: 0.0511 - val_loss: 0.0391
Epoch 8/10
100/100 [==============================] - 158s 2s/step - loss: 0.0458 - val_loss: 0.0429
Epoch 9/10
100/100 [==============================] - 160s 2s/step - loss: 0.0499 - val_loss: 0.0447
Epoch 10/10
100/100 [==============================] - 159s 2s/step - loss: 0.0503 - val_loss: 0.0463


In [24]:
model.save('models/model_2/h5_model_aug.h5')

In [ ]:
cv2.imwrite('compress_img1.png', img,  [cv2.IMWRITE_PNG_COMPRESSION, 9])